# Predicting Housing Prices

## Getting the data:

In [34]:
import pandas as pd
housing = pd.read_csv("C:/Users/Nicoo/Documents/GitHub/Kaggle-Notebooks/data/housing_train.csv")

Remove rows with missing target  and separate target from features:

In [35]:
housing = housing.dropna(axis=0, subset=['SalePrice'])
y = housing.SalePrice
housing = housing.drop(['SalePrice'], axis=1)

Select all columns with non numerical datatype as categorical columns:

In [41]:
categorical = [column for column in housing.columns if housing[column].dtype == "object"]
len(categorical)

43

Selecting numerical columns as well:

In [42]:
numerical = [column for column in housing.columns if housing[column].dtype in ['int64', 'float64']]
len(numerical)

37